## **Preparation of data for plotting**

Here, we consider that all the networks of interest were already generated and saved. This way, we will augment the networks with extra information
and perform extra calculations to generate data ready for plotting.

Let us consider the two types of networks:<br>
1. **Cities flux networks**<br>
2. **Cities to Hospitals flux networks**<br>

## **-1. Lib**

In [2]:
# -- Bib
import os
import sys
sys.path.append("..")

import networkx as nx
import geopandas as gpd
import pandas as pd
import glob
import pickle
from shapely.geometry import Point
import geopy
from geopy.distance import distance, geodesic
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np

from fluxsus.preprocessing.processnet import NetProperties

In [3]:
idx = pd.IndexSlice

## **0. Load base data**

In [4]:
# -- base paths
basepath = os.path.join(os.environ["HOMEPATH"], "Documents", "data")
#basepath = os.path.join(os.environ["HOME"], "Documents", "data")
cnespath = os.path.join(basepath, "opendatasus", "cnes")
geopath = os.path.join(basepath, "shapefilesceqgis")
gmlpath = os.path.join(basepath, "redes_aih")

In [5]:
# -- load geo
geodata_df = gpd.read_parquet(os.path.join(geopath, "ce_geodata.parquet"))
geodata_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 27 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   GEOCOD7            184 non-null    object  
 1   NM_MUNICIP         184 non-null    object  
 2   GEOCOD6            184 non-null    object  
 3   MACRO_ID           184 non-null    int64   
 4   CRES_ID            184 non-null    int64   
 5   geometry_municip   184 non-null    geometry
 6   MACRO_NOME         184 non-null    object  
 7   geometry_macro     184 non-null    geometry
 8   geometry_cres      184 non-null    geometry
 9   MACRO_ID_PROPOSAL  184 non-null    int64   
 10  centroid_municip   184 non-null    geometry
 11  municip_lon        184 non-null    float64 
 12  municip_lat        184 non-null    float64 
 13  2010               184 non-null    float64 
 14  2011               184 non-null    float64 
 15  2012               184 non-null    float64 
 16  

## **1. Maps of proposal**

**Current Macro**

In [6]:
# no preprocessing

**Proposal**

In [7]:
# no preprocessing

## **2. Population, Hospitals and CCA (Original Macro)**

In [14]:
cnes_df = pd.read_parquet(os.path.join(cnespath, "cnes_st_0801_2312.parquet"))
cnes_df = cnes_df.merge(geodata_df[["GEOCOD6", "MACRO_ID", "MACRO_ID_PROPOSAL"]], left_on="CODUFMUN", right_on="GEOCOD6", how="left")

leitos_df = pd.read_parquet(os.path.join(cnespath, "cnes_leitos_timeserie_0801_2312.parquet"))

graph = nx.read_gml(os.path.join(gmlpath, "novo_completo", "citytohospitalnet_agg_1801_2306.gml"))

In [15]:
valid_cnes = []
for v in graph.nodes():
    if graph.nodes[v]['type']=='hospital' and graph.degree(v)>0:
        valid_cnes.append(graph.nodes[v]['code'])

valid_cnes_df = cnes_df[cnes_df["CNES"].isin(valid_cnes)]

In [16]:
# -- geolocation of hospitals
cols = ["CNES", "CODUFMUN", "MACRO_ID", "MACRO_ID_PROPOSAL", "latitude", "longitude", "COMPETEN_MAX", "COMPETEN_MIN"]
valid_cnes_df[cols].to_parquet(os.path.join(gmlpath, "dados_for_plot", "cnes_com_aih_1801_2306.parquet"))

In [17]:
def get_leitos_municipio(municip_code, leitos_df, cnes_df, valid_cnes):
    cnes_of_municipio = cnes_df[(cnes_df["CODUFMUN"]==municip_code) & (cnes_df["CNES"].isin(valid_cnes))]["CNES"].tolist()
    if len(cnes_of_municipio)==0:
        return 0
    else:
        n_leitos_municipio = max(leitos_df.loc[:, idx[cnes_of_municipio, "NUMLEITOS_PRINC"]].sum(axis=1).tolist()[-5:])
        return n_leitos_municipio


muni_leitos = {}
for muni in geodata_df["GEOCOD6"].tolist():
    n_leitos_municipio = get_leitos_municipio(muni, leitos_df, cnes_df, valid_cnes)
    muni_leitos[muni] = n_leitos_municipio

geodata_df["NUMLEITOS"] = geodata_df["GEOCOD6"].map(muni_leitos)
geodata_df[["GEOCOD6", "NUMLEITOS"]].to_parquet(os.path.join(gmlpath, "dados_for_plot", "leitos_por_municipio_1801_2306.parquet"))
geodata_df.head(3)

,GEOCOD7,NM_MUNICIP,GEOCOD6,MACRO_ID,CRES_ID,geometry_municip,MACRO_NOME,geometry_macro,geometry_cres,MACRO_ID_PROPOSAL,...,2015,2016,2017,2018,2019,2020,2021,2022,MACRO_COLOR,NUMLEITOS
0,2300101,ABAIARA,230010,5,19,"POLYGON ((-39.08246 -7.29577, -39.08347 -7.292...",Superintendência Regional de Saúde Cariri,"POLYGON ((-38.67306 -6.70700, -38.67306 -6.707...","POLYGON ((-4354692.823 -865196.973, -4354812.2...",3,...,11259.0,11380.0,11498.0,11619.0,11737.0,11853.0,11965.0,10038.0,#073b4c,0.0
1,2300150,ACARAPE,230015,1,3,"POLYGON ((-38.67268 -4.27393, -38.67268 -4.273...",Superintendência Regional de Saúde Fortaleza,"MULTIPOLYGON (((-39.69667 -2.99902, -39.69216 ...","POLYGON ((-4325426.833 -465433.632, -4325426.8...",1,...,14486.0,14598.0,14707.0,14820.0,14929.0,15036.0,15140.0,14027.0,#ef476f,0.0
2,2300200,ACARAÚ,230020,4,12,"POLYGON ((-39.99113 -3.09797, -39.99117 -3.098...",Superintendência Regional de Saúde Norte,"MULTIPOLYGON (((-39.90892 -3.27414, -39.90892 ...","POLYGON ((-4503430.806 -312287.468, -4503430.7...",2,...,60722.0,61208.0,61679.0,62165.0,62641.0,63104.0,63556.0,64806.0,#118ab2,38.0


### **0.4 CCA for health**

In [145]:
# -- create the two datasets
# -- one: geolocation of health units who genereated an AIH during 2018-2023 (this period can be flexible)
# -- two: geolocation of census units with the number of population contained in each unit
cnes_df = pd.read_parquet(os.path.join(cnespath, "cnes_st_0801_2312.parquet"))
cnes_df = cnes_df[["CNES", "latitude", "longitude"]]

pop_census_df = gpd.read_parquet(os.path.join(geopath, "censo2010_pop_setores.parquet"))
pop_census_df1 = pop_census_df[["CD_GEOCODI", "geometry", "Pop_setor_censo2010"]].copy()
pop_census_df1['centroid'] = pop_census_df1['geometry'].centroid
pop_census_df1 = pop_census_df1.drop('geometry', axis=1).rename({'centroid': 'geometry'}, axis=1).set_geometry('geometry')

# -- the city-hospital bipartite network can provide which health units are actually relevant for analysis (generated at least one AIH during the period chosen)
graph = nx.read_gml(os.path.join(gmlpath, "novo_completo", "citytohospitalnet_agg_1801_2306.gml"))

# -- filter only the relevant health units
valid_cnes = [ graph.nodes[v]['code'] for v in graph.nodes() if graph.nodes[v]['type']=='hospital' and graph.degree(v)>0 ]
valid_cnes_df = cnes_df[cnes_df["CNES"].isin(valid_cnes)]

# -- define the geometry of the health units
valid_cnes_df["geometry"] = gpd.points_from_xy(valid_cnes_df.longitude, valid_cnes_df.latitude)
valid_cnes_df = gpd.GeoDataFrame(valid_cnes_df, geometry='geometry', crs="EPSG:4674")
# -- health units in projection for meters
valid_cnes_df1 = valid_cnes_df.to_crs(epsg=29194).copy()

C:\Users\Higor Monteiro\AppData\Local\Temp\ipykernel_11928\215858703.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pop_census_df1['centroid'] = pop_census_df1['geometry'].centroid
C:\Users\Higor Monteiro\AppData\Local\Temp\ipykernel_11928\215858703.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_cnes_df["geometry"] = gpd.points_from_xy(valid_cnes_df.longitude, valid_cnes_df.latitude)


**Create CCA function (v1)**

In [146]:
# -- basic find and merge procedure
def find_root(index, root_lst):
    temp_index = index
    while root_lst[temp_index]>-1:
        temp_index = root_lst[temp_index]
    return temp_index

def merge_root(root_lst, root1, node1, root2, node2):
    larger_root, larger_node = root1, node1
    smaller_root, smaller_node = root2, node2
    # -- if cluster 2 is larger than cluster 1 (negative scale)
    if root_lst[root1]>root_lst[root2]:
        larger_root, larger_node = root2, node2
        smaller_root, smaller_node = root1, node1

    root_lst[larger_root]+= root_lst[smaller_root]
    root_lst[smaller_root] = larger_root
    return root_lst

def cca_health(l_km, valid_cnes_df, valid_cnes_df1, pop_census_df1):
    '''
    
    '''
    valid_cnes_df1["circle_geometry"] = valid_cnes_df1["geometry"].apply(lambda x: x.buffer(l_km*1000))
    valid_cnes_df1 = valid_cnes_df1.drop('geometry', axis=1).rename({'circle_geometry': 'geometry'}, axis=1)
    valid_cnes_df1 = valid_cnes_df1.to_crs(epsg=4674)

    setor_to_hospital = dict(zip(pop_census_df1["CD_GEOCODI"], [-1 for n in range(pop_census_df1.shape[0]) ]))
    # -- verify which point belongs to which hospital
    for index in range(valid_cnes_df1.shape[0]):
        current_cnes = valid_cnes_df1['CNES'].iloc[index]
        current_polygon = valid_cnes_df1.geometry.iloc[index]

        # -- verification
        cod_setores = pop_census_df1[pop_census_df1.geometry.within(current_polygon)]["CD_GEOCODI"].tolist()
        if len(cod_setores)>0:
            [ setor_to_hospital.update({ current_setor_cod : current_cnes }) for current_setor_cod in cod_setores ]

    # -- verify which hospital is neighbor of which hospital
    hospital_to_hospital = defaultdict(lambda: [])
    for index in range(valid_cnes_df1.shape[0]):
        current_cnes = valid_cnes_df1['CNES'].iloc[index]
        current_polygon = valid_cnes_df1.geometry.iloc[index]
    
        # -- verification
        cod_cnes = valid_cnes_df[valid_cnes_df.geometry.within(current_polygon)]["CNES"].tolist()
        for found_cnes in cod_cnes:
            hospital_to_hospital[current_cnes].append(found_cnes)
    
    root_lst = [ -1 for n in range(len(hospital_to_hospital.keys())) ]
    code_lst = list(sorted(hospital_to_hospital.keys()))
    code_to_index = { cnes: index for index, cnes in enumerate(sorted(hospital_to_hospital.keys())) }

    for n in range(len(root_lst)):
        node_index1 = n
        node_code1 = code_lst[n]
        neighbors_indexes = [ code_to_index[neighbor] for neighbor in hospital_to_hospital[node_code1] ]

        for neighbor in neighbors_indexes:
            root1 = find_root(node_index1, root_lst)
            root2 = find_root(neighbor, root_lst)

            if root1!=root2:
                root_lst = merge_root(root_lst, root1, node_index1, root2, neighbor)
                break

    modules = [ find_root(n, root_lst) for n in range(len(root_lst)) ]
    hospital_to_module = { code_lst[n]: find_root(n, root_lst) for n in range(len(root_lst)) }
    return hospital_to_module, setor_to_hospital

In [147]:
hospital_to_modulel5, setor_to_hospital5 = cca_health(5, valid_cnes_df, valid_cnes_df1, pop_census_df1) # -- 5km
hospital_to_modulel10, setor_to_hospital10 = cca_health(10, valid_cnes_df, valid_cnes_df1, pop_census_df1) # -- 10km
hospital_to_modulel15, setor_to_hospital15 = cca_health(15, valid_cnes_df, valid_cnes_df1, pop_census_df1) # -- 15km
hospital_to_modulel20, setor_to_hospital20 = cca_health(20, valid_cnes_df, valid_cnes_df1, pop_census_df1) # -- 20km
hospital_to_modulel25, setor_to_hospital25 = cca_health(25, valid_cnes_df, valid_cnes_df1, pop_census_df1) # -- 25km

In [148]:
setor_to_hospital10

{'230030925000003': -1,
 '230030925000004': -1,
 '230030925000005': -1,
 '230030925000006': -1,
 '230030925000007': -1,
 '230030925000008': -1,
 '230030925000009': -1,
 '230030925000010': -1,
 '230030925000011': -1,
 '230030925000012': -1,
 '230030930000001': -1,
 '230030930000002': -1,
 '230030930000003': -1,
 '230030930000004': -1,
 '230075428000005': -1,
 '230075430000001': -1,
 '230075430000002': -1,
 '230075430000003': -1,
 '230075430000004': -1,
 '230075435000001': -1,
 '230075435000002': -1,
 '230075440000001': -1,
 '230075440000002': -1,
 '230075440000003': -1,
 '230075440000004': -1,
 '230075440000005': -1,
 '230075440000006': -1,
 '230075440000007': -1,
 '230080405000001': '2563347',
 '230080405000002': '2563347',
 '230100015000012': '2564769',
 '230100015000013': '2564769',
 '230100015000014': '2564769',
 '230100015000015': '2564769',
 '230100015000016': '2564769',
 '230100015000017': '2564769',
 '230100015000018': '2564769',
 '230100015000019': '2564769',
 '230100015000020'

In [359]:
def map_setor_to_cluster(setor_to_hospital, hospital_to_module):
    setor_module = {}
    for setor, cnes in setor_to_hospital.items():
        if cnes!=-1:
            setor_module.update({setor: hospital_to_module[cnes]})
        else:
            setor_module.update({setor: -1})
    return setor_module

In [361]:
setor_module_l5 = map_setor_to_cluster(setor_to_hospital5, hospital_to_modulel5)
setor_module_l10 = map_setor_to_cluster(setor_to_hospital10, hospital_to_modulel10)
setor_module_l15 = map_setor_to_cluster(setor_to_hospital15, hospital_to_modulel15)
setor_module_l20 = map_setor_to_cluster(setor_to_hospital20, hospital_to_modulel20)
setor_module_l25 = map_setor_to_cluster(setor_to_hospital25, hospital_to_modulel25)

In [364]:
pop_census_df["cca_health_module_l5"] = pop_census_df["CD_GEOCODI"].map(setor_module_l5)
pop_census_df["cca_health_module_l10"] = pop_census_df["CD_GEOCODI"].map(setor_module_l10)
pop_census_df["cca_health_module_l15"] = pop_census_df["CD_GEOCODI"].map(setor_module_l15)
pop_census_df["cca_health_module_l20"] = pop_census_df["CD_GEOCODI"].map(setor_module_l20)
pop_census_df["cca_health_module_l25"] = pop_census_df["CD_GEOCODI"].map(setor_module_l25)

In [373]:
valid_cnes_df["cca_health_module_l5"] = valid_cnes_df["CNES"].map(hospital_to_modulel5)
valid_cnes_df["cca_health_module_l10"] = valid_cnes_df["CNES"].map(hospital_to_modulel10)
valid_cnes_df["cca_health_module_l15"] = valid_cnes_df["CNES"].map(hospital_to_modulel15)
valid_cnes_df["cca_health_module_l20"] = valid_cnes_df["CNES"].map(hospital_to_modulel20)
valid_cnes_df["cca_health_module_l25"] = valid_cnes_df["CNES"].map(hospital_to_modulel25)

In [381]:
valid_cnes_df["NUMLEITOS_TOTAL"] = valid_cnes_df["CNES"].apply(lambda x: leitos_df.loc[:, idx[x, 'NUMLEITOS_TODOS']].iat[-1]).fillna(0.0)

In [367]:
pop_census_df.to_parquet(os.path.join(gmlpath, "dados_for_plot", "cca_health_setores.parquet"))

In [383]:
valid_cnes_df.head(5)

,CNES,latitude,longitude,geometry,cca_health_module_l5,cca_health_module_l10,cca_health_module_l15,cca_health_module_l20,cca_health_module_l25,NUMLEITOS_TOTAL
0,9999310,-4.970753,-39.014854,POINT (-39.01485 -4.97075),22,22,22,17,17,0.0
611,9675787,-3.795548,-38.492201,POINT (-38.49220 -3.79555),0,0,0,0,0,2.0
613,9672427,-5.089599,-38.121126,POINT (-38.12113 -5.08960),267,123,122,14,44,142.0
635,9658815,-3.732426,-38.511244,POINT (-38.51124 -3.73243),0,0,0,0,0,4.0
908,9526293,-3.771279,-38.474536,POINT (-38.47454 -3.77128),0,0,0,0,0,25.0


In [384]:
valid_cnes_df.to_parquet(os.path.join(gmlpath, "dados_for_plot", "cca_health_hospitais.parquet"))

### **0.5 Geodesic distances to health services (MACRO ORIGINAL)** 

In [142]:
# -- load geo
geodata_df = gpd.read_parquet(os.path.join(geopath, "ce_geodata.parquet"))

# -- create the two datasets
# -- one: geolocation of health units who genereated an AIH during 2018-2023 (this period can be flexible)
# -- two: geolocation of census units with the number of population contained in each unit
cnes_df = pd.read_parquet(os.path.join(cnespath, "cnes_st_0801_2312.parquet"))
cnes_df = cnes_df[["CNES", "CODUFMUN", "latitude", "longitude"]]
cnes_df = cnes_df.merge(geodata_df[["GEOCOD6", "MACRO_ID"]], left_on="CODUFMUN", right_on="GEOCOD6", how="left").drop("GEOCOD6", axis=1)

pop_census_df = gpd.read_parquet(os.path.join(geopath, "censo2010_pop_setores.parquet"))
pop_census_df1 = pop_census_df[["CD_GEOCODI", "CD_GEOCODM", "geometry", "Pop_setor_censo2010"]].copy()
pop_census_df1["GEOCOD6"] = pop_census_df1["CD_GEOCODM"].apply(lambda x: x[:6])
pop_census_df1 = pop_census_df1.merge(geodata_df[["GEOCOD6", "MACRO_ID"]], on="GEOCOD6", how="left")
pop_census_df1['centroid'] = pop_census_df1['geometry'].centroid
pop_census_df1 = pop_census_df1.drop('geometry', axis=1).rename({'centroid': 'geometry'}, axis=1).set_geometry('geometry')
pop_census_df1 = pop_census_df1.to_crs(epsg=29194).copy()

# -- the city-hospital bipartite network can provide which health units are actually relevant for analysis (generated at least one AIH during the period chosen)
graph = nx.read_gml(os.path.join(gmlpath, "novo_completo", "citytohospitalnet_agg_1801_2306.gml"))

# -- filter only the relevant health units
valid_cnes = [ graph.nodes[v]['code'] for v in graph.nodes() if graph.nodes[v]['type']=='hospital' and graph.degree(v)>0 ]
valid_cnes_df = cnes_df[cnes_df["CNES"].isin(valid_cnes)]

# -- define the geometry of the health units
valid_cnes_df["geometry"] = gpd.points_from_xy(valid_cnes_df.longitude, valid_cnes_df.latitude)
valid_cnes_df = gpd.GeoDataFrame(valid_cnes_df, geometry='geometry', crs="EPSG:4674")
# -- health units in projection for meters
valid_cnes_df1 = valid_cnes_df.to_crs(epsg=29194).copy()
#valid_cnes_df1 = valid_cnes_df.copy()

C:\Users\Higor Monteiro\AppData\Local\Temp\ipykernel_11928\1632097510.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pop_census_df1['centroid'] = pop_census_df1['geometry'].centroid
C:\Users\Higor Monteiro\AppData\Local\Temp\ipykernel_11928\1632097510.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_cnes_df["geometry"] = gpd.points_from_xy(valid_cnes_df.longitude, valid_cnes_df.latitude)


In [143]:
pop_census_df1.to_crs(epsg=29194)

,CD_GEOCODI,CD_GEOCODM,Pop_setor_censo2010,GEOCOD6,MACRO_ID,geometry
0,230030925000003,2300309,377.0,230030,5,POINT (476989.139 9330948.729)
1,230030925000004,2300309,199.0,230030,5,POINT (480048.176 9336260.657)
2,230030925000005,2300309,636.0,230030,5,POINT (485252.963 9327763.325)
3,230030925000006,2300309,437.0,230030,5,POINT (490192.900 9332257.138)
4,230030925000007,2300309,409.0,230030,5,POINT (476086.976 9327061.471)
...,...,...,...,...,...,...
13610,231410225000006,2314102,1207.0,231410,4,POINT (265605.153 9592798.508)
13611,231410225000007,2314102,622.0,231410,4,POINT (270411.386 9593609.425)
13612,231410225000008,2314102,474.0,231410,4,POINT (272069.444 9595249.363)
13613,231410225000009,2314102,283.0,231410,4,POINT (268797.885 9596696.820)


In [122]:
macro_id = 2
subset_cnes = valid_cnes_df1[valid_cnes_df1["MACRO_ID"]==macro_id]
subset_pop_setor = pop_census_df1[pop_census_df1["MACRO_ID"]==macro_id]

for index in range(subset_cnes.shape[0]):
    current_point_cnes = subset_cnes.geometry.iloc[index]

macro_distances = subset_cnes.geometry.apply(lambda g: subset_pop_setor.distance(g)/1000).values.flatten() # km

In [125]:
current_point_cnes = subset_cnes.geometry.iloc[0]

In [139]:
arr = (subset_pop_setor.distance(current_point_cnes)/1000)*subset_pop_setor["Pop_setor_censo2010"].apply(lambda x: np.ones(int(x)) if pd.notna(x) else np.array([]))

In [141]:
arr.apply(list).sum()

[82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,
 82.27692298734185,


In [124]:
subset_cnes.geometry.apply(lambda g: subset_pop_setor.distance(g)/1000)

,45,46,47,48,49,50,51,52,53,54,...,13007,13008,13009,13010,13011,13012,13013,13014,13015,13016
0,82.276923,91.346129,98.910696,98.076564,97.028125,99.435137,74.377453,75.576503,70.341660,67.653217,...,205.479591,199.307094,190.550705,192.555260,183.996308,200.337526,206.692975,216.784222,189.855320,193.722059
4630,53.574464,57.411655,78.583783,78.137377,77.887143,77.324667,57.463835,60.458917,51.415085,45.697196,...,170.248832,164.002751,154.870944,157.018179,148.367803,164.634689,171.161186,181.338603,154.068833,158.239651
13096,36.743553,15.468479,66.931459,67.523930,69.083126,62.212897,64.278120,68.498625,59.022618,52.633465,...,111.762009,105.465992,96.213846,98.363956,89.702901,105.985924,112.505819,122.709387,95.472093,99.603349
13173,126.037830,146.927063,113.088443,111.782720,109.410096,117.773867,98.462169,95.026113,101.815348,107.002596,...,250.299154,245.363874,241.541746,241.638761,234.919295,250.837352,254.542964,262.889625,242.412037,242.096255
13197,107.368410,118.335853,119.536511,118.539818,117.126723,120.946302,94.797226,95.022582,92.008966,90.790835,...,232.799319,226.676475,218.153862,220.070312,211.571449,227.950991,234.196938,244.228984,217.524835,221.203240
13263,116.150807,136.330928,106.632928,105.322180,103.008929,110.950718,89.513206,86.460252,92.110837,96.652090,...,242.760493,237.603255,233.054787,233.417875,226.376516,242.504362,246.614741,255.280356,233.716609,233.978787
13276,66.413747,52.519540,98.756196,98.949531,99.847878,95.096512,86.899073,91.018812,80.395939,73.066170,...,142.341844,135.871273,124.324328,127.548857,118.531691,133.532992,141.222575,151.695848,122.542380,129.126878
13301,93.088987,92.968149,118.598277,118.111697,117.764857,117.438058,96.645002,99.236308,90.918058,85.609599,...,199.590335,193.160439,182.433466,185.293176,176.333837,191.896919,199.240512,209.673565,180.966578,186.748585
13768,160.041969,142.645551,171.547210,172.813846,175.203399,166.509600,185.765648,188.892143,183.475379,179.835849,...,36.900172,39.886253,42.241020,42.060813,48.946784,32.815566,30.271435,25.683449,41.915239,41.752177
13770,165.339744,143.580149,188.517494,189.547545,191.708058,183.127171,193.701173,197.704783,189.199033,183.335712,...,94.293154,91.808497,81.215582,85.489081,83.658579,80.061344,86.264380,90.743569,77.447170,86.996513


array([156.1513061 , 167.09107654, 163.78022816, ..., 118.07668637,
       113.58981223, 114.10074978])

## **0. Time series of hospitals and hospital beds per macro**

In [ ]:
cnes_df = pd.read_parquet(os.path.join(cnespath, "cnes_st_0801_2312.parquet"))
leitos_df = pd.read_parquet(os.path.join(cnespath, "cnes_leitos_timeserie_0801_2312.parquet")).fillna(0)

# -- get info on macro for each CNES
cnes_macro = cnes_df.merge(geodata_df[["MACRO_ID", "MACRO_ID_PROPOSAL", "GEOCOD6"]], left_on="CODUFMUN", right_on="GEOCOD6", how="left")[["CNES", "CODUFMUN", "MACRO_ID", "MACRO_ID_PROPOSAL"]]

In [ ]:
# -- objective: for each macro, sum the number of hospital beds of the hospitals inside the macro
def count_hospitalbed_per_macro(leitos_df, macro_id, cnes_macro, macro_col="MACRO_ID"):
    ''' 
        Return the number of hospital beds (main and all) for the macrorregion for each time
        point.
    '''
    cnes_list = cnes_macro[cnes_macro[macro_col]==macro_id]["CNES"].tolist()
    subset_pri =  leitos_df.loc[:, idx[cnes_list, 'NUMLEITOS_PRINC']].sum(axis=1)
    subset_todos =  leitos_df.loc[:, idx[cnes_list, 'NUMLEITOS_TODOS']].sum(axis=1)
    macro_res = pd.DataFrame({
        "period": leitos_df.index, "macro_leitos_princ": subset_pri, "macro_leitos_todos": subset_todos
    })
    return macro_res

macro_original = { n : count_hospitalbed_per_macro(leitos_df, n, cnes_macro, macro_col="MACRO_ID") for n in range(1,5+1)  }
macro_proposal = { n : count_hospitalbed_per_macro(leitos_df, n, cnes_macro, macro_col="MACRO_ID_PROPOSAL") for n in range(1,8+1)  }

In [ ]:
macro_original = pd.concat([ macro_original[key] for key in macro_original.keys() ], axis=1, keys=macro_original.keys())
macro_proposal = pd.concat([ macro_proposal[key] for key in macro_proposal.keys() ], axis=1, keys=macro_proposal.keys())


In [ ]:
macro_original.to_parquet(os.path.join(gmlpath, "dados_for_plot", "timeserie_macro_original_beds.parquet"))
macro_proposal.to_parquet(os.path.join(gmlpath, "dados_for_plot", "timeserie_macro_proposal_beds.parquet"))


In [ ]:
macro_original.head()

1                                            2                     \
     period macro_leitos_princ macro_leitos_todos period macro_leitos_princ   
0801   0801             6406.0             9918.0   0801              499.0   
0802   0802             6450.0             9962.0   0802              499.0   
0803   0803             6450.0             9971.0   0803              499.0   
0804   0804             6611.0            10148.0   0804              499.0   
0805   0805             6632.0            10192.0   0805              499.0   

                             3                                            4  \
     macro_leitos_todos period macro_leitos_princ macro_leitos_todos period   
0801              882.0   0801              546.0              901.0   0801   
0802              882.0   0802              546.0              901.0   0802   
0803              884.0   0803              546.0              908.0   0803   
0804              884.0   0804              552.0              914.0   0804   
0805              884.0   0805              552.0              914.0   0805   

                                                5                     \
     macro_leitos_princ macro_leitos_todos period macro_leitos_princ   
0801             1389.0             2414.0   0801             2180.0   
0802             1389.0             2412.0   0802             2186.0   
0803             1389.0             2418.0   0803             2192.0   
0804             1362.0             2388.0   0804             2189.0   
0805             1365.0             2391.0   0805             2184.0   

                         
     macro_leitos_todos  
0801             3362.0  
0802             3368.0  
0803             3383.0  
0804             3371.0  
0805             3367.0

In [ ]:
macro_proposal.head()

1                                            2                     \
     period macro_leitos_princ macro_leitos_todos period macro_leitos_princ   
0801   0801             6111.0             9337.0   0801              520.0   
0802   0802             6155.0             9381.0   0802              520.0   
0803   0803             6155.0             9390.0   0803              520.0   
0804   0804             6315.0             9565.0   0804              497.0   
0805   0805             6352.0             9633.0   0805              480.0   

                             3                                            4  \
     macro_leitos_todos period macro_leitos_princ macro_leitos_todos period   
0801             1031.0   0801             1606.0             2325.0   0801   
0802             1031.0   0802             1607.0             2326.0   0802   
0803             1037.0   0803             1613.0             2337.0   0803   
0804             1005.0   0804             1613.0             2330.0   0804   
0805              980.0   0805             1613.0             2330.0   0805   

      ...                  5      6                                        \
      ... macro_leitos_todos period macro_leitos_princ macro_leitos_todos   
0801  ...              721.0   0801              558.0              929.0   
0802  ...              721.0   0802              558.0              929.0   
0803  ...              723.0   0803              558.0              936.0   
0804  ...              723.0   0804              564.0              942.0   
0805  ...              723.0   0805              564.0              942.0   

          7                                            8                     \
     period macro_leitos_princ macro_leitos_todos period macro_leitos_princ   
0801   0801              386.0              623.0   0801              855.0   
0802   0802              386.0              622.0   0802              855.0   
0803   0803              386.0              622.0   0803              855.0   
0804   0804              386.0              629.0   0804              852.0   
0805   0805              390.0              633.0   0805              852.0   

                         
     macro_leitos_todos  
0801             1474.0  
0802             1473.0  
0803             1473.0  
0804             1470.0  
0805             1470.0  

[5 rows x 24 columns]

## **1. Between cities flux networks**

### **1.1 Mapping and macros**

In [ ]:
netsuffix = "agg_2001_2012"

# -- load and perform calculations
fname = f"cityfluxnet_{netsuffix}.gml"
graph = nx.read_gml(os.path.join(gmlpath, "novo_completo", fname))
netprop = NetProperties(graph).calculate_in_flow(weight_people_col='admission_count', weight_cost_col='total_cost').calculate_out_flow(weight_people_col='admission_count', weight_cost_col='total_cost')
netprop.process_infomap_graph().process_louvain_graph()
graph = netprop.graph

# -- complementary info on graph (macro proposal)
#temp_ = dict(zip(geodata_df["GEOCOD6"], geodata_df["MACRO_ID_PROPOSAL"]))
#for v in graph.nodes():
#    graph.nodes[v]['macro_id_proposal'] = temp_[graph.nodes[v]['municipio_code']]

# -- add modules info on the geodataframe
geodata_df1 = geodata_df.copy()
geodata_df1["infomap_count_module"] = geodata_df1["GEOCOD6"].map({graph.nodes[u]['municipio_code']: graph.nodes[u]['infomap_count_module_id'] for u in graph.nodes()})
geodata_df1["infomap_cost_module"] = geodata_df1["GEOCOD6"].map({graph.nodes[u]['municipio_code']: graph.nodes[u]['infomap_cost_module_id'] for u in graph.nodes()})
geodata_df1["louvain_count_module"] = geodata_df1["GEOCOD6"].map({graph.nodes[u]['municipio_code']: graph.nodes[u]['louvain_count_module_id'] for u in graph.nodes()})
geodata_df1["louvain_cost_module"] = geodata_df1["GEOCOD6"].map({graph.nodes[u]['municipio_code']: graph.nodes[u]['louvain_cost_module_id'] for u in graph.nodes()})
geodata_df1["inflow_people"] = geodata_df1["GEOCOD6"].map({graph.nodes[u]['municipio_code']: graph.nodes[u]['incoming_people'] for u in graph.nodes()})
geodata_df1["inflow_cost"] = geodata_df1["GEOCOD6"].map({graph.nodes[u]['municipio_code']: graph.nodes[u]['incoming_cost'] for u in graph.nodes()})

found 9 modules with codelength: 2.5192578014211713
found 3 modules with codelength: 2.4760044338086376


In [ ]:
# -- specify base colors
# ---- original colors based on specific cities
# ---- new modules will be colored with the original colors if one of the cities below is included

cmap_macro_original = {1: "#ef476f", 2: "#ffb300", 3: "#04ae81", 4: "#118ab2", 5: "#073b4c"}

citycode_colors = {"230440": "#ef476f", "230730": "#073b4c", "230410": "#118ab2", "231140": "#ffb300", "231180": "#04ae81"}
citylabel_colors = {"58": "#ef476f", "98": "#073b4c", "49": "#118ab2", "150": "#ffb300", "154": "#04ae81"}
extra_colors = ["#00756a", "#bcb20f", "#ff8239", "#c6881c", "#00a2a3", "#4d4d4d", "#f0e2e7", "#dbfe87", "#1c448e"]

# -- create new cmap for new modules
#cmap_infomap_count, cmap_infomap_cost = {}, {}
#cmap_louvain_count, cmap_louvain_cost = {}, {}

def get_cmap(graph, citycode_colors, extra_colors, property_name):
    cmap = {}
    n_modules = len(set( [ graph.nodes[v][property_name] for v in graph.nodes() ] ))
    for v in graph.nodes():
        if graph.nodes[v]['municipio_code'] in citycode_colors.keys():
            if graph.nodes[v][property_name] not in cmap.keys():
                cmap.update({graph.nodes[v][property_name] : citycode_colors[graph.nodes[v]['municipio_code']]})
    extra_colors_ = extra_colors[:] + [ x for x in citycode_colors.values() if x not in cmap.values() ]
    dummy_index = 0
    for n in range(n_modules):
        if n+1 not in cmap.keys():
            cmap.update({ n+1: extra_colors_[dummy_index]})
            dummy_index+=1
    return cmap

cmap_11 = get_cmap(graph, citycode_colors, extra_colors, 'infomap_count_module_id')
cmap_13 = get_cmap(graph, citycode_colors, extra_colors, 'infomap_cost_module_id')
cmap_21 = get_cmap(graph, citycode_colors, extra_colors, 'louvain_count_module_id')
cmap_23 = get_cmap(graph, citycode_colors, extra_colors, 'louvain_cost_module_id')
#cmap_prop = get_cmap(graph, citycode_colors, extra_colors, 'macro_id_proposal')


geodata_df1["INFOMAP_COUNT_COLOR"] = geodata_df1["infomap_count_module"].map(cmap_11)
geodata_df1["INFOMAP_COST_COLOR"] = geodata_df1["infomap_cost_module"].map(cmap_13)
geodata_df1["LOUVAIN_COUNT_COLOR"] = geodata_df1["louvain_count_module"].map(cmap_21)
geodata_df1["LOUVAIN_COST_COLOR"] = geodata_df1["louvain_cost_module"].map(cmap_23)
#geodata_df1["MACRO_PROPOSAL_COLOR"] = geodata_df1["MACRO_ID_PROPOSAL"].map(cmap_prop)

In [ ]:
cmap_11

{7: '#118ab2',
 1: '#ef476f',
 2: '#073b4c',
 9: '#04ae81',
 3: '#00756a',
 4: '#bcb20f',
 5: '#ff8239',
 6: '#c6881c',
 8: '#00a2a3'}

In [ ]:
geodata_df1 = geodata_df1.drop(["centroid_municip"], axis=1)
geodata_df1.to_parquet(os.path.join(gmlpath, "dados_for_plot", f"geoforplot_{netsuffix}.parquet"))

In [ ]:
geodata_df1.columns

Index(['GEOCOD7', 'NM_MUNICIP', 'GEOCOD6', 'MACRO_ID', 'CRES_ID',
       'geometry_municip', 'MACRO_NOME', 'geometry_macro', 'geometry_cres',
       'MACRO_ID_PROPOSAL', 'municip_lon', 'municip_lat', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022', 'MACRO_COLOR', 'infomap_count_module',
       'infomap_cost_module', 'louvain_count_module', 'louvain_cost_module',
       'inflow_people', 'inflow_cost', 'INFOMAP_COUNT_COLOR',
       'INFOMAP_COST_COLOR', 'LOUVAIN_COUNT_COLOR', 'LOUVAIN_COST_COLOR'],
      dtype='object')

### **1.1 time series networks - Fluxes over time**

In [ ]:
fnames = sorted(glob.glob(os.path.join(gmlpath, "novo_completo", "*noverlap*")))
list_of_networks = [ nx.read_gml(fname) for fname in fnames ]

**Define utility functions**

Calculate the incoming, outcoming and internal flows for each macrorregion.

In [ ]:
def get_fluxes_macro(graph, macro_indices=["1", "2", "3", "4", "5"]):
    '''
        Calculate the incoming and outgoing fluxes for each macro. Also calculate the internal flux. 
    '''
    # -- initialize flows dict.
    macro_in_out = {
        ind: {
            "INCOMING": 0,
            "OUTGOING": 0,
            "INTERNAL": 0
        } for ind in macro_indices
    }

    # -- calculation
    for src, tgt in graph.edges():
        cur_action = graph.edges[(src, tgt)]
        
        src_macro, tgt_macro = str(int(cur_action['source_macro'])), str(int(cur_action['target_macro']))
        flux_count = cur_action['admission_count']

        if src_macro==tgt_macro:
            macro_in_out[src_macro]["INTERNAL"] += flux_count
        else:
            macro_in_out[src_macro]["OUTGOING"] += flux_count
            macro_in_out[tgt_macro]["INCOMING"] += flux_count
    return macro_in_out

def macro_flows_time(list_of_graphs, macro_indices=["1", "2", "3", "4", "5"]):
    ''' 
    
    '''
    # -- initialize flows dict.
    macro_in_out = {
        ind: {
            "INCOMING": [],
            "OUTGOING": [],
            "INTERNAL": []
        } for ind in macro_indices
    }
    for graph in list_of_graphs:
        res = get_fluxes_macro(graph)
        for key in res.keys():
            macro_in_out[key]["INCOMING"].append(res[key]["INCOMING"])
            macro_in_out[key]["OUTGOING"].append(res[key]["OUTGOING"])
            macro_in_out[key]["INTERNAL"].append(res[key]["INTERNAL"])
    return macro_in_out

In [ ]:
macro_flows = macro_flows_time(list_of_networks)
with open(os.path.join(gmlpath, 'dados_for_plot', 'macro_original_fluxes.pickle'), 'wb') as handle:
    pickle.dump(macro_flows, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import graph_tool as gt
from graph_tool.inference import minimize_blockmodel_dl

In [ ]:
fname = "cityfluxnet_agg_1801_2212.gml"
graph = gt.load_graph(os.path.join(gmlpath, "novo_completo", fname))

In [ ]:
state = minimize_blockmodel_dl(graph)
modules_v = state.get_blocks()

In [ ]:
# YYMM
fname = "cityfluxnet_agg_1801_2212.gml"
graph = nx.read_gml(os.path.join(gmlpath, "novo_completo", fname))

In [ ]:
for v in graph.nodes():
    break
graph.nodes[v]

{'municipio_code': '230010',
 'municipio_name': 'ABAIARA',
 'macro_id': 5,
 'macro_new_id': 3,
 'macro_name': 'Superintendência Regional de Saúde Cariri',
 'cres_id': 19,
 'lat': -7.35990681440754,
 'lon': -39.03753839591732}

In [ ]:
for u, v in graph.edges():
    break
graph.edges[(u,v)]

{'admission_count': 27,
 'total_cost': 16354.65,
 'source_macro': 5.0,
 'target_macro': 5,
 'source_micro': 19.0,
 'target_micro': 19,
 'same_macro': 5.0,
 'same_micro': 19.0,
 'admission_count_ch1': 0.0,
 'total_cost_ch1': 0.0,
 'admission_count_ch2': 0.0,
 'total_cost_ch2': 0.0,
 'admission_count_ch3': 0.0,
 'total_cost_ch3': 0.0,
 'admission_count_ch4': 0.0,
 'total_cost_ch4': 0.0,
 'admission_count_ch5': 0.0,
 'total_cost_ch5': 0.0,
 'admission_count_ch6': 0.0,
 'total_cost_ch6': 0.0,
 'admission_count_ch7': 0.0,
 'total_cost_ch7': 0.0,
 'admission_count_ch8': 0.0,
 'total_cost_ch8': 0.0,
 'admission_count_ch9': 0.0,
 'total_cost_ch9': 0.0,
 'admission_count_ch10': 0.0,
 'total_cost_ch10': 0.0,
 'admission_count_ch11': 6.0,
 'total_cost_ch11': 3420.04,
 'admission_count_ch12': 0.0,
 'total_cost_ch12': 0.0,
 'admission_count_ch13': 1.0,
 'total_cost_ch13': 948.13,
 'admission_count_ch14': 6.0,
 'total_cost_ch14': 2934.94,
 'admission_count_ch15': 14.0,
 'total_cost_ch15': 9051.54,
 

## **2. Cities to Hospitals flux networks**